# Modify a standard Grid road network
#### Delete some intersections, move intersections / roads spatially and define an empty trafficlight dict (use Phase.ipynb to define new phases)

In [1]:
import json

In [2]:
def create_trafficlights(lenght, virtual):
    if virtual: 
        trafficLight = {
            "roadLinkIndices": list(range(lenght)),
            "lightphases": [
                {
                    "time": 30,
                    "availableRoadLinks": [
                    ]
                },
                {
                    "time": 30,
                    "availableRoadLinks": []
                }
            ]
        }
        
    else:
        trafficLight = {
            "roadLinkIndices": list(range(lenght)),
            "lightphases": [
                {
                    "time": 30,
                    "availableRoadLinks": [
                        0
                    ]
                },
                {
                    "time": 30,
                    "availableRoadLinks": []
                }
            ]
        }
    
    return trafficLight

In [3]:
def choose_del_intersections():
    del_intersections = set()

    for i in range(11, 30):
        for e in range(7, 9):
            del_intersections.add(f"intersection_{i}_{e}")
            
    return del_intersections

In [4]:
def delete_intersections(data):
    # which intersections to delete
    del_intersections = choose_del_intersections()
    print(f"number of deleted intersections {len(del_intersections)}")

    counter = 0

    adjacent_roads = set()

    # First round to find all roads that will be deleted
    del_indices = []

    for index, intersection in enumerate(data["intersections"]):

        if intersection["id"] in del_intersections:

            # Get all adjacent roads
            adjacent_roads.update(set(intersection["roads"]))

            del_indices.append(index)

            counter += 1

    for index in del_indices[::-1]:
        del data["intersections"][index]

    # Second round to delete all roads from all intersections
    for intersection in data["intersections"]:

        # Delete roads
        intersection["roads"] = [road for road in intersection["roads"] if road not in adjacent_roads]

        # Delete roadLinks
        intersection["roadLinks"] = [roadLink for roadLink in intersection["roadLinks"] if
                                     roadLink["endRoad"] not in adjacent_roads and roadLink["startRoad"] not in adjacent_roads]

    # Delete all roads from data["roads"]
    data["roads"] = [road for road in data["roads"] if road["id"] not in adjacent_roads]


    # Adjust all traffic lights and delete unncessary intersections
    del_indices = []

    for index, intersection in enumerate(data["intersections"]):

        # number of incoming lanes
        number_incoming = len(intersection["roadLinks"])

        virtual = intersection["virtual"]

        # create some sample lightphase
        intersection["trafficLight"] = create_trafficlights(number_incoming, virtual)

        # delete unncessary intersections
        if len(intersection["roads"]) == 0:
               del_indices.append(index)

    for index in del_indices[::-1]:
        del data["intersections"][index]


    counter = 0
    for inter in data["intersections"]:
        if inter["virtual"] == False:
            counter += 1

    print(f"There are {counter} intersections left in the network")



In [5]:
def move_intersections_and_lanes(data, axis, selective_shifting):
    
    save_shift = {}
    
    base = 0
    end = 0
    
    for select, amount in selective_shifting.items():
        
        for intersection in data["intersections"]:

            index = int(intersection["id"].split("_")[1]) if axis == "x" else int(intersection["id"].split("_")[2]) if axis == "y" else None
            
            if index <= select[0] or index > select[1]:
                continue
                
                
            if axis == "y":
                shift = (index - end + 1) * amount + base

            elif axis == "x":
                shift = (index - end + 1) * amount + base
            
            save_shift[intersection["id"]] = shift

            # move intersection id's points
            intersection["point"][axis] += shift

            # move roadlinks points
            if intersection["roadLinks"]:
                for roadLink in intersection["roadLinks"]:
                    for laneLinks in roadLink["laneLinks"]:
                        for point in laneLinks["points"]:
                            point[axis] += shift
                
                
        base += shift
        end = select[1] + 1
        
            
            
    for road in data["roads"]:
        
        # start point
        road["points"][0][axis] += save_shift[road["startIntersection"]]
        
        # end point
        road["points"][1][axis] += save_shift[road["endIntersection"]]
        

In [6]:
# Opening JSON file
json_file = "../Simulation_Results/man/"

with open(json_file + "roadnet.json") as f:
    data = json.load(f)
    
    
move_intersections_and_lanes(data, "x", {(-1, 31): -200})
move_intersections_and_lanes(data, "y", {(-1, 6): -150, (6, 12): -30})


delete_intersections(data)


# Save the modified data as a new JSON file
new_json_file = json_file + "roadnet_modified.json"
with open(new_json_file, "w") as f:
    json.dump(data, f, indent=4)

print(f"Modified data saved as {new_json_file}")

number of deleted intersections 38
There are 292 intersections left in the network
Modified data saved as ../Simulation_Results/man/roadnet_modified.json
